In [42]:
import os
from llama_index.llms.perplexity import Perplexity
from dotenv import load_dotenv

load_dotenv()

pplx_api_key = os.getenv("PERPLEXITY_API_KEY")
# os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_KEY')

llm = Perplexity(
    api_key=pplx_api_key, model="sonar-small-online", temperature=0.5
)

In [43]:
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.core import StorageContext

username = "neo4j"
password = os.getenv('NEO4J_PW')
url = os.getenv('NEO4J_URL')
database = "neo4j"

graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)


In [61]:
from llama_index.core.indices.knowledge_graph.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from entity_extraction.entity_extraction import custom_entity_extract_fn

graph_rag_retriever_with_nl2graphquery = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
    entity_extract_fn=custom_entity_extract_fn,
    with_nl2graphquery=True
)

query_engine_with_nl2graphquery = RetrieverQueryEngine.from_args(
    graph_rag_retriever_with_nl2graphquery,
)

In [47]:
from llama_index.core.llms import ChatMessage

query = "who is harry potter"

pre_response = query_engine_with_nl2graphquery.query(
    query,
)

if pre_response.metadata is None:
    # 1) answer needs to be queried from Llama2 endpoint
    messages_dict = [
        {"role": "system", "content": "Be precise and concise."},
        {"role": "user", "content": query},
    ]
    messages = [ChatMessage(**msg) for msg in messages_dict]

    external_response = llm.chat(messages)
    print(external_response)

    # 2) answer needs to be stored into txt file for loading into KG
    with open('data/external_response.txt', 'w') as f:
        print(external_response, file=f)
else:
    print("From Graph: " + str(pre_response))
    print(list(list(pre_response.metadata.values())[0]["kg_rel_map"].keys()))

assistant: Harry Potter is a fictional character created by British author J.K. Rowling. He is the protagonist of a series of seven fantasy novels that follow his life as a young wizard attending Hogwarts School of Witchcraft and Wizardry. Harry is raised by his aunt and uncle after losing his parents at a young age, and he discovers his magical abilities at the age of 11. Throughout the series, Harry faces challenges, makes friends, and battles against the dark wizard Lord Voldemort. The character has become iconic and has inspired multiple adaptations into film, stage productions, and theme park attractions.


In [49]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=['data/external_response.txt']
).load_data()

index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=5,
)

In [64]:
post_response = query_engine_with_nl2graphquery.retrieve(
    "who is harry potter",
)

print(post_response)

[]
